In [1]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from itertools import islice, takewhile, chain
from functools import reduce
from typing import Optional
import datetime as dt
from dataclasses import asdict, fields
from importlib import reload

from geopy.distance import distance
from shapely.geometry import Point, LineString
import shapely.geometry as sg
import geopandas as gpd

from typing import List
import ipyleaflet as lf

pd.set_option('display.max_rows', 1000000000)

In [2]:
import busboy.model as m
import busboy.geo as geo
import busboy.database as db
import busboy.prediction as prediction
import busboy.map.map as bmap
import busboy.apis as api
import busboy.util as util
import busboy.util.notebooks as notebook

/Users/Noel/.local/share/virtualenvs/Busboy-8t8akAoa/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [50]:
reload(util)
reload(geo)
reload(m)
reload(db)
reload(prediction)
reload(bmap)
reload(api)
reload(notebook)

<module 'busboy.util.notebooks' from '/Users/Noel/Developer/Projects/Busboy/busboy/util/notebooks.py'>

In [3]:
rbn = db.routes_by_name()
route = rbn["220"].id
entries = db.snapshots(r=route, d=dt.date(2019, 2, 18))
stops_by_name = db.stops_by_name()
timetable_variants = {t for timetable in timetables for t in timetable.variants}

In [4]:
entries_by_vehicle = util.dict_collect_list(entries, lambda e: e.vehicle)
vehicles_by_entry_count = [v for (v, es) in sorted(entries_by_vehicle.items(), key = lambda t: len(t[1]))]

In [5]:
pvars = sorted(list(
    prediction.possible_variants(
        prediction.drop_duplicate_positions(entries_by_vehicle[vehicles_by_entry_count[10]]), 
        timetable_variants
    )),
    key = lambda t: t[0].poll_time)

In [6]:
order_pvars = list(prediction.check_variant_order(pvars))

AttributeError: 'Nothing' object has no attribute 'get'

In [84]:
precheck = [(len(t[1]), t[0].latitude, t[0].longitude, t[0].poll_time.time().isoformat(), {(tv.route, i, tv.stops[i].name, tv.stops[i + 1].name) for (tv, i) in t[1]}) for t in islice(pvars, 200, 300)]
precheck

[(6,
  51.87828416666667,
  -8.436893055555556,
  '20:01:27.593951',
  {('220',
    12,
    'Douglas Village East (Shopping Centre)',
    'Douglas Road (South Link Exit Slip Road)'),
   ('220',
    18,
    'Douglas Village East (Shopping Centre)',
    'Douglas Road (South Link Exit Slip Road)'),
   ('220',
    21,
    'Douglas Village East (Shopping Centre)',
    'Douglas Road (South Link Exit Slip Road)'),
   ('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (6,
  51.87699805555555,
  -8.435628888888889,
  '20:01:47.597543',
  {('220',
    11,
    'Maryborough Hill (Paddocks)',
    'Douglas Village East (Shopping Centre)'),
   ('220',
    17,
    'Maryborough Hill (Paddocks)',
    'Douglas Village East (Shopping Centre)'),
   ('220',
    20,
    'Maryborough Hill (Paddocks)',
    'Douglas Village East (Shopping Centre)'),
   ('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (6,
  51.8

In [85]:
postcheck = [(len(t[1]), t[0].latitude, t[0].longitude, t[0].poll_time.time().isoformat(), {(tv.route, i, tv.stops[i].name, tv.stops[i + 1].name) for (tv, i) in t[1]}) for t in islice(order_pvars, 200, 300)]
postcheck

[(3,
  51.87828416666667,
  -8.436893055555556,
  '20:01:27.593951',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:01:47.597543',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:02:07.608235',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:02:27.624700',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:02:47.641593',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:03:07.658397',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East

In [22]:
# show stop times
# collected = [util.dict_collect_set(vs, lambda tpl: tpl[0]) for (e, vs) in order_pvars]
stop_shaped_entries = [(e, {v: {t[1] for t in ts}}) for (e, vs) in order_pvars for (v, ts) in util.dict_collect_set(vs, lambda tpl: tpl[0]).items()]
times = prediction.stop_times(stop_shaped_entries)

In [23]:
calculated_times = set()

for v, ts in islice(times.items(), 0, 10):
    for p, stop_times in islice(ts.items(), 0, 100):
        for t in stop_times:
            calculated_times.add(tuple(chain([p, v.stops[p].name], (x for x in t))))
            
            
for (position, name, t1, t2) in sorted(calculated_times, key=lambda tpl: tpl[2]):
    print(f"{position:3}: {name:50} ({t1.time().isoformat()} -> {t2.time().isoformat()})")

  8: Carrigaline (Carrigaline Court Hotel)              (02:37:15.314657 -> 02:37:35.333903)
  2: Carrigaline (Carrigaline Court Hotel)              (02:37:15.314657 -> 02:37:35.333903)
 11: Carrigaline (Carrigaline Court Hotel)              (02:37:15.314657 -> 02:37:35.333903)
 12: Carrigaline (Cork Rd Opp Dentist)                  (02:37:35.333903 -> 02:37:55.347708)
  3: Carrigaline (Cork Rd Opp Dentist)                  (02:37:35.333903 -> 02:37:55.347708)
  9: Carrigaline (Cork Rd Opp Dentist)                  (02:37:35.333903 -> 02:37:55.347708)
  4: Carrigaline (Glenview)                             (02:38:15.351298 -> 02:38:35.354742)
 10: Carrigaline (Glenview)                             (02:38:15.351298 -> 02:38:35.354742)
 13: Carrigaline (Glenview)                             (02:38:15.351298 -> 02:38:35.354742)
 11: Carrigaline (Opp Herons Wood)                      (02:38:55.358394 -> 02:39:15.362066)
  5: Carrigaline (Opp Herons Wood)                      (02:38:55.3583

In [40]:
themap = bmap.Map()
themap.map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [43]:
timetables208 = list(api.timetables("208", stops_by_name))
timetables205 = list(api.timetables("205", stops_by_name))

In [46]:
notebook.show_timetables(themap, timetables208)

In [49]:
notebook.show_timetables(themap, timetables205)

In [30]:
notebook.plot_entries(
    themap, 
    (t[0] for t in islice(stop_shaped_entries, 400, 600)), 
    snapshot_to_layer = lambda e: lf.Marker(
        location=(e.latitude, e.longitude),
        draggable=False,
        title=e.poll_time.time().isoformat()
    ),
    clear=False
)

In [7]:
vehicles_by_entry_count

[VehicleId(raw='7338674957838188756'),
 VehicleId(raw='7338674957838188752'),
 VehicleId(raw='7338674957838188925'),
 VehicleId(raw='7338674957838188758'),
 VehicleId(raw='7338674957838188929'),
 VehicleId(raw='7338674957838188928'),
 VehicleId(raw='7338674957838189413'),
 VehicleId(raw='7338674957838188953'),
 VehicleId(raw='7338674957838188804'),
 VehicleId(raw='7338674957838188930'),
 VehicleId(raw='7338674957838189374'),
 VehicleId(raw='7338674957838188809'),
 VehicleId(raw='7338674957838188917'),
 VehicleId(raw='7338674957838188920'),
 VehicleId(raw='7338674957838188879'),
 VehicleId(raw='7338674957838189409'),
 VehicleId(raw='7338674957838189406'),
 VehicleId(raw='7338674957838188838'),
 VehicleId(raw='7338674957838189385'),
 VehicleId(raw='7338674957838189370'),
 VehicleId(raw='7338674957838188805'),
 VehicleId(raw='7338674957838188762'),
 VehicleId(raw='7338674957838188973'),
 VehicleId(raw='7338674957838188927'),
 VehicleId(raw='7338674957838188918'),
 VehicleId(raw='733867495